# PostgreSQL with Python

This is the study note written by __Siyu Yao (April)__ on 220809 - 220812.

Focus on following things:
- [Link Postgres database to python](#link-to-database)
- Use python to do database CRUD ([create](#create-table), [read](#fetch-data-query-data), [update](#update-table), [delete](#delete-data))
- [Join query, Group By, Order By, Aggregate Functions](#joins)
- [Views](#views)

## Link  To Database

For note organization, following code is with out **try-except**, for full version, see below Final Full Version.

In [1]:
#import library work with PostgreSQL
import psycopg2
import psycopg2.extras        # used for context manager, see below

In [2]:
#the needed information for link db, can be found in postico login page

hostname = 'localhost'
database = 'mydb'
username = 'i52'
pwd = ''
port_id = 5432

In [3]:
#to record connection and cursor statement, see later

conn = None
cur = None

In [4]:
#link to datebase
conn = psycopg2.connect(host = hostname,
                        dbname = database,
                        user = username,
                        password = pwd,
                        port = port_id)

In [5]:
#create a 'cursor' object for actually wotking with qeries

cur = conn.cursor()     #nomal way
#cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)     #in dictionary from

print('Database connected.\n')

Database connected.



## Drop View

In [6]:
#drop before create to avoid duplicate in multi-time pratice execute
cur.execute('DROP VIEW IF EXISTS myview')

print('View dropped.\n')

View dropped.



## Drop Table

In [7]:
#drop before create to avoid duplicate in multi-time pratice execute
cur.execute('DROP TABLE IF EXISTS employee')
cur.execute('DROP TABLE IF EXISTS department')
cur.execute('DROP TABLE IF EXISTS manager')
cur.execute('DROP TABLE IF EXISTS projects')
cur.execute('DROP TABLE IF EXISTS family')

print('Table dropped.\n')

Table dropped.



## Commit Change

In [8]:
#only after change commited, the db would change, otherwise the python script would not affect db.
conn.commit()

## Create Table

In [9]:
#create table
create_script = '''CREATE TABLE IF NOT EXISTS employee(     --only create if the table not created before
                        emp_id      varchar(20) PRIMARY KEY,
                        name        varchar(50) NOT NULL, 
                        salary      int,                    -- inside is part of SQL, comment style change
                        dept_id     varchar(20),
                        manager_id  varchar(20)
                        )'''       #same syntax as in SQL
#execute the change
cur.execute(create_script)

In [10]:
#create other tables for later use
create_script = '''CREATE TABLE IF NOT EXISTS department(
                        dept_id   varchar(20),
                        dept_name varchar(50)
                        )'''
cur.execute(create_script)

create_script = '''CREATE TABLE IF NOT EXISTS manager(
                        manager_id   varchar(20),
                        manager_name varchar(50),
                        dept_id      varchar(20)
                        )'''
cur.execute(create_script)

create_script = '''CREATE TABLE IF NOT EXISTS projects(
                        project_id     varchar(20),
                        project_name   varchar(100),
                        team_member_id varchar(20)
                        )'''
cur.execute(create_script)

create_script = '''CREATE TABLE IF NOT EXISTS family(
                        member_id varchar(10),
                        name      varchar(50),
                        age       int,
                        parent_id varchar(10));'''
cur.execute(create_script)

conn.commit()
print('Table created.\n')

Table created.



## Insert Date Into DB

In [11]:
#insert date into db
insert_script = '''INSERT INTO employee (emp_id, name, salary, dept_id, manager_id)
                    VALUES (%s, %s, %s, %s, %s)'''
insert_values = [('E1', 'James', 8000, 'D1', 'M1'), ('E2', 'Bob', 9500, 'D1', 'M1'), 
                ('E3', 'Eva', 7000, 'D1', 'M2'), ('E4', 'Jack', 13000, 'D1', 'M2'),
                ('E5', 'Robin', 20000,'D10', 'M3'), ('E6', 'Amy', 15000, 'D10', 'M3'),
                ('E7', 'May', 5000, 'D3', 'M3')]


for value in insert_values:
        cur.execute(insert_script, value)

In [12]:
conn.commit()

In [13]:
insert_script = '''INSERT INTO department (dept_id, dept_name)
                    VALUES (%s, %s)'''
insert_values = [('D1', 'IT'), ('D2', 'HR'),
                 ('D3', 'Finance'), ('D4', 'Admin')]
for value in insert_values:
        cur.execute(insert_script, value)


insert_script = '''INSERT INTO manager(manager_id, manager_name, dept_id)
                    VALUES(%s, %s, %s)'''
insert_values = [('M1', 'Prem', 'D3'), ('M2', 'Shripadh', 'D4'),
                 ('M3', 'Nick', 'D1'), ('M4', 'Cory', 'D1')]
for value in insert_values:
        cur.execute(insert_script, value)


insert_script = '''INSERT INTO projects(project_id, project_name, team_member_id)
                     VALUES(%s, %s, %s)'''
insert_values = [('P1', 'Data Migration', 'E1'), ('P1', 'Data Migration', 'E2'),
                 ('P1', 'Data Migration', 'M3'),
                 ('P2', 'ETL Tool', 'E1'), ('P2', 'ETL Tool', 'M4')]
for value in insert_values:
        cur.execute(insert_script, value)
        
insert_script = '''INSERT INTO family(member_id, name, age, parent_id)
                     VALUES(%s, %s, %s, %s)'''
insert_values = [('F1', 'David', 4, 'F5'), ('F2', 'Carol', 10, 'F5'),
                 ('F3', 'Michael', 12,  'F5'),('F4', 'Johnson', 36, 'NULL'), 
                 ('F5', 'Maryam', 40, 'F6'),('F6', 'Stewart', 70, 'NULL'), 
                 ('F7', 'Rohan', 6, 'F4'),('F8', 'Asha', 8, 'F4')]
for value in insert_values:
        cur.execute(insert_script, value)

## Fetch Data/ Query Data

In [14]:
#fetch data from db
cur.execute('SELECT * FROM employee')

#if only have one record can use fetchone()

print(cur.fetchall())     #print in list
print('---------- This is print in list. ----------\n')


cur.execute('SELECT * FROM employee')
for record in cur.fetchall():
    print(record)     #one at a line in tuple
print('---------- This is print at a line in tuple. ----------\n')



cur.execute('SELECT * FROM employee')
for record in cur.fetchall():   
    print(record[1], record[2])     #print only name and salary based on index
print('---------- This is print only name and salary based on index. ----------\n')
    
    
    #print(record['name'], record['salary'])		#use cursor dictionary form
    #print('---------- This is print using cursor dictionary form. ----------\n')


# conn.commit()

[('E1', 'James', 8000, 'D1', 'M1'), ('E2', 'Bob', 9500, 'D1', 'M1'), ('E3', 'Eva', 7000, 'D1', 'M2'), ('E4', 'Jack', 13000, 'D1', 'M2'), ('E5', 'Robin', 20000, 'D10', 'M3'), ('E6', 'Amy', 15000, 'D10', 'M3'), ('E7', 'May', 5000, 'D3', 'M3')]
---------- This is print in list. ----------

('E1', 'James', 8000, 'D1', 'M1')
('E2', 'Bob', 9500, 'D1', 'M1')
('E3', 'Eva', 7000, 'D1', 'M2')
('E4', 'Jack', 13000, 'D1', 'M2')
('E5', 'Robin', 20000, 'D10', 'M3')
('E6', 'Amy', 15000, 'D10', 'M3')
('E7', 'May', 5000, 'D3', 'M3')
---------- This is print at a line in tuple. ----------

James 8000
Bob 9500
Eva 7000
Jack 13000
Robin 20000
Amy 15000
May 5000
---------- This is print only name and salary based on index. ----------



## Update Table

In [15]:
#update table
update_script = 'UPDATE employee SET salary = 1.2 * salary'

cur.execute(update_script)

In [16]:
cur.execute('SELECT * FROM employee')
for record in cur.fetchall():
    print(record)     #one at a line in tuple

('E1', 'James', 9600, 'D1', 'M1')
('E2', 'Bob', 11400, 'D1', 'M1')
('E3', 'Eva', 8400, 'D1', 'M2')
('E4', 'Jack', 15600, 'D1', 'M2')
('E5', 'Robin', 24000, 'D10', 'M3')
('E6', 'Amy', 18000, 'D10', 'M3')
('E7', 'May', 6000, 'D3', 'M3')


## Delete Data

In [17]:
delete_script = 'DELETE FROM employee WHERE name = %s'
delete_record = ('May',)

cur.execute(delete_script, delete_record)

In [18]:
cur.execute('SELECT * FROM employee')
for record in cur.fetchall():
    print(record)     #one at a line in tuple

('E1', 'James', 9600, 'D1', 'M1')
('E2', 'Bob', 11400, 'D1', 'M1')
('E3', 'Eva', 8400, 'D1', 'M2')
('E4', 'Jack', 15600, 'D1', 'M2')
('E5', 'Robin', 24000, 'D10', 'M3')
('E6', 'Amy', 18000, 'D10', 'M3')


In [19]:
#only after change commited, the db would change, otherwise the python script would not affect db.
conn.commit()

## Joins
Use joins to combine separate tables' information into one. DO NOT create new relationship but simply providing the result for query. For creating new relationship quicker for next query, see 'Views'.

In following, LHS = left hand side, RHS = right hand side. The L/R based on relevent location of ___ JOIN.

### INNER JOIN / JOIN
Inner join only fetch the records which are present in **both side** of table under the join codition.<br>
The two key word in SQL, INNER JOIN also write as JOIN for short since it's the most frequence use.

Task: Fetch the *employee name* and the *department name* they belong to.

In [20]:
inner_script = '''SELECT e.name, d.dept_name
                    FROM employee AS e             -- LHS table
                    INNER JOIN department AS d     -- RHS table
                    ON e.dept_id = d.dept_id;      -- join condition, behind 'ON'
                    '''   
cur.execute(inner_script)

In [21]:
for record in cur.fetchall():
    print(record)

('Jack', 'IT')
('Eva', 'IT')
('Bob', 'IT')
('James', 'IT')


### LEFT JOIN / LEFT OUTER JOIN

**LHS** table take priority and become main table.  <br>
Left join = inner join + any additional records in the LHS table.

Task: Fetch **ALL the *employee name*** and the *department name* they belong to.

In [22]:
left_script = '''SELECT e.name, d.dept_name
                    FROM employee AS e             -- LHS table
                    LEFT JOIN department AS d     -- RHS table
                    ON e.dept_id = d.dept_id;      -- join condition
                    '''   
cur.execute(left_script)

In [23]:
for record in cur.fetchall():
    print(record)

('Jack', 'IT')
('Eva', 'IT')
('Bob', 'IT')
('James', 'IT')
('Amy', None)
('Robin', None)


### RIGHT JOIN / RIGHT OUTER JOIN

**RHS** table take priority and become main table.  <br>
Right join = inner join + any additional records in the RHS table.

Task: Fetch the *employee name* and **ALL the *department name*** they belong to.

In [24]:
left_script = '''SELECT e.name, d.dept_name
                    FROM employee AS e             -- LHS table
                    RIGHT JOIN department AS d     -- RHS table
                    ON e.dept_id = d.dept_id;      -- join condition
                    '''   
cur.execute(left_script)

In [25]:
for record in cur.fetchall():
    print(record)

('Jack', 'IT')
('Eva', 'IT')
('Bob', 'IT')
('James', 'IT')
(None, 'HR')
(None, 'Finance')
(None, 'Admin')


### FULL JOIN / FULL OUTER JOIN

Full join = inner join + all remaining records from LHS + all remaining records from RHS

Task: Fetch **ALL the *employee name*** and **ALL the *department name*** they belong to.

In [26]:
full_script = '''SELECT e.name, d.dept_name
                    FROM employee AS e             -- LHS table
                    FULL JOIN department AS d      -- RHS table
                    ON e.dept_id = d.dept_id;      -- join condition, behind 'ON'
                    '''   
cur.execute(full_script)

In [27]:
for record in cur.fetchall():
    print(record)

('Jack', 'IT')
('Eva', 'IT')
('Bob', 'IT')
('James', 'IT')
(None, 'HR')
(None, 'Finance')
(None, 'Admin')
('Amy', None)
('Robin', None)


### CROSS JOIN / CARTESIAN JOIN

Returns cartesian product(every record in LHS will match every record in RHS).  

Task: Fetch ALL possible match for employee name and department name.

In [28]:
cross_script = '''SELECT e.name, d.dept_name
                    FROM employee AS e             -- LHS table
                    CROSS JOIN department AS d      -- RHS table
                    -- cross join  do not need join condition
                    '''   
cur.execute(cross_script)

In [29]:
for record in cur.fetchall():
    print(record)

('James', 'IT')
('Bob', 'IT')
('Eva', 'IT')
('Jack', 'IT')
('Robin', 'IT')
('Amy', 'IT')
('James', 'HR')
('Bob', 'HR')
('Eva', 'HR')
('Jack', 'HR')
('Robin', 'HR')
('Amy', 'HR')
('James', 'Finance')
('Bob', 'Finance')
('Eva', 'Finance')
('Jack', 'Finance')
('Robin', 'Finance')
('Amy', 'Finance')
('James', 'Admin')
('Bob', 'Admin')
('Eva', 'Admin')
('Jack', 'Admin')
('Robin', 'Admin')
('Amy', 'Admin')


### NATURAL JOIN

Sometimes looks like inner join, but nutural join is decided by SQL not by user, it may become a problem. 

SQL go through each column's name in specified table, if table share same column name, SQL use that column to join tables together. If not find the dame column name, SQL will do the cross join.

Let SQL to take control rather then yourself, highly not recommend.

In [30]:
natural_script = '''SELECT e.name, d.dept_name
                    FROM employee AS e             -- LHS table
                    NATURAL JOIN department AS d      -- RHS table
                    -- natural join  do not need join condition
                    '''   
cur.execute(natural_script)

In [31]:
for record in cur.fetchall():
    print(record)

('Jack', 'IT')
('Eva', 'IT')
('Bob', 'IT')
('James', 'IT')


### SELF JOIN

Join the table to itself. Self join do not has its own keyword, use table twice is self join. Self join can use keyword join/ left join/ right join ... as the keyword normal affect.

Task: Fetch the child name and their age corresponding to their parent name and parent age

In [32]:
self_script = '''SELECT child.name AS child_name, child.age AS child_age,
                    parent.name AS parent_name, parent.age AS parent_age 
                    FROM family AS child            -- LHS table
                    JOIN family AS parent           -- RHS table
                    ON child.parent_id = parent.member_id    -- join condition
                    '''   
cur.execute(self_script)

In [33]:
#conn.commit()

In [34]:
for record in cur.fetchall():
    print(record)

('David', 4, 'Maryam', 40)
('Carol', 10, 'Maryam', 40)
('Michael', 12, 'Maryam', 40)
('Maryam', 40, 'Stewart', 70)
('Rohan', 6, 'Johnson', 36)
('Asha', 8, 'Johnson', 36)


## VIEWS

Main purpose of using vies:
- To simplfiy complex SQL queries. 
    - When you need to get information from many tables at once, write query everytime become time-consuming. 
- Security.
    - Or you want to share people only know the basic information not the top secret. 

You can create view to represent this relationship. View is a database object, created over an SQL query, does NOT store any data, more like a virtual table.

Task: Fetch details of ALL emp name, their manager, their department and the projects they working on.

In [35]:
multi_script = '''SELECT e.name, d.dept_name, m.manager_name, p.project_name
                    FROM employee AS e
                    LEFT JOIN department AS d ON e.dept_id = d.dept_id
                    INNER JOIN manager AS m ON m.manager_id = e.manager_id
                    LEFT JOIN projects AS p ON p.team_member_id = e.emp_id;
                ''' # the order of the join is like ((e LEFT JOIN d) INNER JOIN m) LEFT JOIN p
cur.execute(multi_script)

In [36]:
for record in cur.fetchall():
    print(record)

('James', 'IT', 'Prem', 'ETL Tool')
('Bob', 'IT', 'Prem', 'Data Migration')
('James', 'IT', 'Prem', 'Data Migration')
('Eva', 'IT', 'Shripadh', None)
('Jack', 'IT', 'Shripadh', None)
('Amy', None, 'Nick', None)
('Robin', None, 'Nick', None)


### Create View

In [37]:
view_script = '''CREATE VIEW all_in_one AS
                    SELECT e.name, d.dept_name, m.manager_name, p.project_name
                    FROM employee AS e
                    LEFT JOIN department AS d ON e.dept_id = d.dept_id
                    INNER JOIN manager AS m ON m.manager_id = e.manager_id
                    LEFT JOIN projects AS p ON p.team_member_id = e.emp_id;
                ''' # the order of the join is like ((e LEFT JOIN d) INNER JOIN m) LEFT JOIN p
cur.execute(view_script)

In [38]:
cur.execute('SELECT * FROM all_in_one')

for record in cur.fetchall():
    print(record)

('James', 'IT', 'Prem', 'ETL Tool')
('Bob', 'IT', 'Prem', 'Data Migration')
('James', 'IT', 'Prem', 'Data Migration')
('Eva', 'IT', 'Shripadh', None)
('Jack', 'IT', 'Shripadh', None)
('Amy', None, 'Nick', None)
('Robin', None, 'Nick', None)


### CREATE OR REPLACE 

Using CREATE OR REPLACE keyword, SQL will check if the view is already create, if no then create one, if created, replace the old one with the new.

Rules when using  CREATE OR REPLACE:
- CANNOT change the column name.
- CANNOT change the column data type.
- CANNOT change th order of the column. (but can add new column at the end)

In [39]:
cor_script = '''CREATE OR REPLACE VIEW all_in_one AS
                    SELECT e.name, d.dept_name, m.manager_name, p.project_name, p.project_id
                    FROM employee AS e
                    LEFT JOIN department AS d ON e.dept_id = d.dept_id
                    INNER JOIN manager AS m ON m.manager_id = e.manager_id
                    LEFT JOIN projects AS p ON p.team_member_id = e.emp_id;
                ''' # the order of the join is like ((e LEFT JOIN d) INNER JOIN m) LEFT JOIN p
cur.execute(cor_script)

In [40]:
cur.execute('SELECT * FROM all_in_one')

for record in cur.fetchall():
    print(record)

('James', 'IT', 'Prem', 'ETL Tool', 'P2')
('Bob', 'IT', 'Prem', 'Data Migration', 'P1')
('James', 'IT', 'Prem', 'Data Migration', 'P1')
('Eva', 'IT', 'Shripadh', None, None)
('Jack', 'IT', 'Shripadh', None, None)
('Amy', None, 'Nick', None, None)
('Robin', None, 'Nick', None, None)


### Modifiyng a View
- Change the column 
- Rename the view
- Drop the view
- Refresh the view
    - Insert new data in table will auto added in view.
    - Data strcture (added new column in table)would NOT auto refresh  -> use CREATE OR REPLACE to refresh the view

In [41]:
# change the column name
# same as change the column name in table
# only affect view, not affect original table
cur.execute('ALTER VIEW all_in_one RENAME COLUMN name TO employee_name')


# rename the view
cur.execute('ALTER VIEW all_in_one RENAME TO all_in_one_v2')

# drop the view
cur.execute('DROP VIEW all_in_one_v2')

### Updateble Views

Views which can be UPDATE/ INSERTED/ DELETED

Rules:
- View should be created using 1 table/ view only
    - When update view, the original table will also be updated (since view was created from the table)
- View query cannot have DISTINCT clause
- View query connot have GROUP BY clause
- View query connot have WITH clause
- View query connot have window functions

### WITH CHECK OPTION

Once view is created, if want to insert into the view, SQL will check if the insert data is satisfying the WHERE condition. If not satisfying will throw an error.

In [42]:
check_script = ''' CREATE OR REPLACE VIEW myview AS
                    SELECT * FROM employee WHERE dept_id = 'D10'
                    WITH CHECK OPTION;
               '''

cur.execute(check_script)


cur.execute('SELECT * FROM myview')
for record in cur.fetchall():
    print(record)

#cur.execute('INSERT INTO myview (emp_id, name, salary, dept_id, manager_id) VALUES ('E8', 'Padma', 10000, 'D10', 'M2');')

('E5', 'Robin', 24000, 'D10', 'M3')
('E6', 'Amy', 18000, 'D10', 'M3')


In [43]:
conn.commit()

## Disconnect Database

In [44]:
#colse the conncetion if conncet success
if cur is not None:
    cur.close()
if conn != None:
    conn.close()
print("Conncetion closed!")

Conncetion closed!


## Final Full Version

Connect and disconnect to DB, CRUD, with context manager and try except.

In [45]:
import psycopg2
import psycopg2.extras

hostname = 'localhost'
database = 'mydb'
username = 'i52'
pwd = ''
port_id = 5432

conn = None
cur = None


try:
    with psycopg2.connect(host = hostname,
                        dbname = database,
                        user = username,
                        password = pwd,
                        port = port_id) as conn:

        with conn.cursor(cursor_factory = psycopg2.extras.DictCursor) as cur:
            #drop before create to avoid duplicate in multi-time pratice execute
            cur.execute('DROP VIEW IF EXISTS myview')
            cur.execute('DROP TABLE IF EXISTS employee')


            #create table
            create_script = '''CREATE TABLE IF NOT EXISTS employee(     --only create if the table not created before
                        emp_id      varchar(20) PRIMARY KEY,
                        name        varchar(50) NOT NULL, 
                        salary      int,                    -- inside is part of SQL, comment style change
                        dept_id     varchar(20),
                        manager_id  varchar(20)
                        )'''       #same syntax as in SQL
            #execute the change
            cur.execute(create_script)


            #insert date into db
            insert_script = '''INSERT INTO employee (emp_id, name, salary, dept_id, manager_id)
                    VALUES (%s, %s, %s, %s, %s)'''
            insert_values = [('E1', 'James', 8000, 'D1', 'M1'), ('E2', 'Bob', 9500, 'D1', 'M1'), 
                            ('E3', 'Eva', 7000, 'D1', 'M2'), ('E4', 'Jack', 13000, 'D1', 'M2'),
                            ('E5', 'Robin', 20000,'D10', 'M3'), ('E6', 'Amy', 15000, 'D10', 'M3'),
                            ('E7', 'May', 5000, 'D3', 'M3')]
            for value in insert_values:
                    cur.execute(insert_script, value)


            #update table
            update_script = 'UPDATE employee SET salary = 1.2 * salary'
            cur.execute(update_script)

            #delete data
            delete_script = 'DELETE FROM employee WHERE name = %s'
            delete_record = ('Jack',)
            cur.execute(delete_script, delete_record)


            #fetch data from db
            cur.execute('SELECT * FROM employee')
            for record in cur.fetchall():
                print(record['name'], record['salary'])     #use cursor dictionary form

            #no conn.commit() becaue manager will take care     

except Exception as e:
    print('Something wrong with connect to DB')
    print(e)
finally:
    #manager will take care of closing cursor
    if conn != None:
        conn.close()    #still need to close db manually




James 9600
Bob 11400
Eva 8400
Robin 24000
Amy 18000
May 6000
